In [4]:
#transform function for brookshire. the problem i need to solve is with the store_week. the file does not have any data 
#in regards to the date. This would cause risk in duplicate data error  neeed to solve in the future.

import re
import pandas as pd
import datetime as dt
import psycopg2

import pandas.io.sql as psql

import numpy as np
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

store_type_input = 'brookshire'
file = 'winwin_details'
transition_date_range = '2022SS'
current_year = 2022
current_week = 19
store_week = ''

connection = psycopg2.connect(database=f"{store_type_input}", user="postgres", password="winwin", host="localhost")

old = pd.read_excel(f'{file}.xlsx', skiprows=2)


#drops uneccessary collumns
old = old.drop(columns=['Pack','Size'])

#drops rows that dont have any important data
old = old[old["Description"].str.contains("Totals for Store Number:")==False]

old.drop(old.tail(1).index, inplace=True)  # drop last n rows


old_len = len(old)

old[['abc','store_number']] = old["Description"].str.split(":", 1, expand=True)

old = old.drop(columns=['abc', 'Description'])
#if it contains brookshire bros then grab the store number then go to next row
#          if not add number
i=0
x=0

# setting store numbers to rows
while i < old_len:
    
    #selecting store number
    cell =old.iloc[i,3]
    
    if cell == None:
        
        old.iloc[i,3] = store_number
        
    
    elif '#' in cell:
        
        store_number = cell[1:6]
        store_number = int(store_number)

        
    else:
        
        print('Broken', i)
        
    i+=1
    
    
#filtering all sales data has $0  
filt = (old['Sales $'] != 0)

old = old.loc[filt]


#drop ros that contain NaN
old = old.dropna()



#rename columns
old = old.rename(columns={
    
    'Store': 'store_number',
    'UPC': 'upc',
    'Sales $': 'sales',
    'Units Sold': 'qty',
    
    })

old['transition_date_range'] = transition_date_range
old['current_year'] = current_year
old['current_week'] = current_week
old['store_year'] = current_year
old['store_week'] = store_week
old['store_type'] = 'brookshire'

salesdata = old[['transition_date_range',
                 'store_year',
                 'store_week',
                 'store_number',
                 'upc',
                 'sales',
                 'qty',
                 'current_year',
                 'current_week',
                 'store_type']]

salesdata[['upc', 'qty']] = salesdata[['upc', 'qty']].astype(np.int64)


In [6]:
salesdata.dtypes



transition_date_range     object
store_year                 int64
store_week                 int64
store_number              object
upc                        int64
sales                    float64
qty                        int64
current_year               int64
current_week               int64
store_type                object
dtype: object

In [12]:
#select max date
#use max date to calc 3 weeks back to find min date
#use dates for deliv reference
#use the 2 sql statment and use a except function
    

import re
import pandas as pd
from datetime import datetime, timedelta
import psycopg2

import pandas.io.sql as psql

import numpy as np
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

store_type_input = 'fresh_encounter'


connection = psycopg2.connect(database=f"{store_type_input}", user="postgres", password="winwin", host="localhost")    
    
    
    
max_date = psql.read_sql('select max(date) from delivery', connection)
max_date = max_date.iloc[0,0]
min_date = max_date - timedelta(days=21)



no_scan = psql.read_sql(f"""

/*select all of the items that have 0 sales  from the the sd_combo table*/

SELECT store, item_group_desc 
FROM SD_COMBO 
WHERE (season = 'AY' or season = 'SS') and 
       total_sales = 0 

/*except statement is taking out any item that are duplicates*/
EXCEPT

    /*Select statement is grabbing all of the items that have been shipped in the past 3 weeks*/

    select store, item_group_desc from delivery
    inner join item_support on delivery.upc = item_support.upc
    where date <= '{max_date}' and date >= '{min_date}'

order by store

""", connection)


2022-04-22 2022-05-13


In [8]:
# from store_info import Replenishment

# jewel = Replenishment('brookshire','2022SS', '2022', '19')
# # delta.delivery_import('kroger_delta_deliverydata')
# # delta.delivery_update('kroger_central')
# # delta.sales_import('kroger_delta_salesdata')
# jewel.sales_update('winwin_details')



In [17]:
#transform jewel program

import re
import pandas as pd
import datetime as dt
import psycopg2

import pandas.io.sql as psql

import numpy as np
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

store_type_input ='jewel'
file = '2022 Winwin Sales Data'
transition_date_range = '2022SS'
current_week = 20
current_year = 2022
connection = psycopg2.connect(database=f"{store_type_input}", user="postgres", password="winwin", host="localhost")    


old = pd.read_excel(f'{file}.xlsx', sheet_name='Product Scan', skiprows=1)

# inserting neccessary collumns for sales insert function
old['transition_date_range'] = transition_date_range
old['current_year'] = current_year
old['current_week'] = current_week

# grabbing the date to get the store year

date = old.iloc[0, 3]
store_year = date.year
old['store_year'] = store_year

# Grabs only the neccessary collumns and reformats them into the neccesary df format for the sale insertert function
old = old[['transition_date_range',
           'store_year',
           'Day',
           'Store',
           'UPC',
           'Sum Net Amount',
           'Sum Item Quantity',
           'current_year',
           'current_week',
           'Division']]

old = old.rename(columns={
    'Division': 'store_type',
    'Store': 'store_number',
    'UPC': 'upc',
    'Sum Net Amount': 'sales',
    'Sum Item Quantity': 'qty',
    'Day': 'store_week'
})

# extracting only the store division name from colllumn and then lower caseing it so it can pass the db security check
old['store_type'] = old.store_type.str.replace('[^a-zA-Z]', '')
old['store_type'] = old.store_type.str.lower()

# Find the last date in the db

date = psql.read_sql('select max(store_week) from sales', connection)
date = pd.Timestamp(date.iloc[0, 0])

old.head(50)

# sorts dates in order and then drops current week so current week can be added based on store_week
old = old.sort_values(by='store_week', ascending=True)

old = old.drop(columns=['current_week'])

# using that date select the data from that date to present day in the sales sheet.

filt = (old['store_week'] >= date)

old = old.loc[filt]

# uses to the dates from store_week and finds their week number they are on and converts data typed
old[['year', 'current_week', 'day']] = old['store_week'].dt.isocalendar()

old['current_week'] = old['current_week'].astype(np.int64)


#adding one to the current_week bc the data that was used from excel before we transition to the database was using the 
#the weeknum fucntion in excel. To stay consistent with the week numbers, I added one to each of the current week.
# After that, the if code that follows is checking to make sure the store_week num is between 1-52



old_len = len(old)

while i < old_len:

    store_week = old.iloc[i, 12]
    
    if store_week < 
    
    



# drops unneccesary collumns and reorders
# old = old.drop(columns=['year', 'day'])



# salesdata = old[['transition_date_range',
#                  'store_year',
#                  'store_week',
#                  'store_number',
#                  'upc',
#                  'sales',
#                  'qty',
#                  'current_year',
#                  'current_week',
#                  'store_type']]
# salesdata
old

C:\Users\User1\AppData\Local\Temp/ipykernel_3192/1059202128.py:55: FutureWarning: The default value of regex will change from True to False in a future version.
  old['store_type'] = old.store_type.str.replace('[^a-zA-Z]', '')


,transition_date_range,store_year,store_week,store_number,upc,sales,qty,current_year,store_type,year,current_week,day
10729,2022SS,2022,2022-05-22,3178,81031202372,39.98,2,2022,jewel,2022,21,7
10722,2022SS,2022,2022-05-22,2160,81031202374,19.99,1,2022,jewel,2022,21,7
10723,2022SS,2022,2022-05-22,3051,81031202441,19.99,1,2022,jewel,2022,21,7
10724,2022SS,2022,2022-05-22,3051,81031202065,7.99,1,2022,jewel,2022,21,7
10725,2022SS,2022,2022-05-22,3051,81031202369,7.99,1,2022,jewel,2022,21,7
...,...,...,...,...,...,...,...,...,...,...,...,...
10662,2022SS,2022,2022-05-22,3484,81031202447,79.96,4,2022,jewel,2022,21,7
10663,2022SS,2022,2022-05-22,4057,81031202366,23.97,3,2022,jewel,2022,21,7
10664,2022SS,2022,2022-05-22,4057,81031202368,0.00,0,2022,jewel,2022,21,7
10654,2022SS,2022,2022-05-22,3341,81031202418,19.99,1,2022,jewel,2022,21,7


In [44]:
#trying to fix week num problem for jewel

from datetime import datetime, timedelta

old = pd.read_excel('jewel_salesdata.xlsx')
old = old.drop(columns=['current_week'])


#creating a collumn where 1 day is subtracted so week num is calculated, weeknum starts from monday not sunday
old['start_date'] = old['store_week'] - timedelta(days=1)

old[['year', 'current_week', 'day']] = old['start_date'].dt.isocalendar()

old['current_week'] = old['current_week'] + 1

old = old.drop(columns =['year','day'])
# old.to_excel('jewel_replacement.xlsx', index = False)

old

# dateTimeInstance = datetime.datetime(2022, 1, 1)
# weeknum =dateTimeInstance.isocalendar()
# weeknum

,transition_date_range,store_year,store_week,store_number,upc,sales,qty,current_year,store_type,start_date,current_week
0,NaN,2021,2021-02-28,52,81031202046,6.99,1,2021,jewel,2021-02-27,9
1,NaN,2021,2021-02-28,185,81031202111,13.00,4,2021,jewel,2021-02-27,9
2,NaN,2021,2021-02-28,3084,81031202111,12.98,2,2021,jewel,2021-02-27,9
3,NaN,2021,2021-03-01,3426,81031202046,6.99,1,2021,jewel,2021-02-28,9
4,NaN,2021,2021-03-01,3426,81031202065,34.95,5,2021,jewel,2021-02-28,9
...,...,...,...,...,...,...,...,...,...,...,...
104063,2022SS,2022,2022-05-22,3191,81031202418,19.99,1,2022,jewel,2022-05-21,21
104064,2022SS,2022,2022-05-22,3191,81031202420,39.98,2,2022,jewel,2022-05-21,21
104065,2022SS,2022,2022-05-22,3741,81031202420,19.99,1,2022,jewel,2022-05-21,21
104066,2022SS,2022,2022-05-22,3770,81031202420,19.99,1,2022,jewel,2022-05-21,21


In [1]:
# support sheet import revamp

from store_info import *


delta = Replenishment(store_type_input='kvat',
                               transition_date_range='2022SS', 
                               current_year = 2022, 
                               current_week= 21)

delta.delivery_import('kvat_deliverydata_fix')


NumericValueOutOfRange: integer out of range
